In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.utils import save_image
import sys
import os
import cv2
from PIL import Image
from tqdm import tqdm
from random import randint
import matplotlib.pyplot as plt
import pandas as pd
import math
import glob
torch.cuda.empty_cache()

weights_path = 'C:/Users/vanag/Desktop/classification_model-3.pkl'
classifier_path = 'C:/Users/vanag/Desktop'

annotations_dir = 'C:/Users/vanag/Desktop/BPTI/datasets/drone-tracking-datasets/annotations_dataset2.json'
files_dir = 'C:/Users/vanag/Desktop/BPTI/datasets/drone-tracking-datasets/dataset2'

batch_size = 64

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
sys.path.append(classifier_path)
import zoomed_classifier as classifier
model = classifier.create_classification_model(weights_path, device)

In [ ]:
def get_square_bboxes(bboxes):
    xmin, ymin, xmax, ymax = bboxes[0], bboxes[1], bboxes[2], bboxes[3]

    if (xmax - xmin) > (ymax - ymin):
      a, b = ymax, ymin
      ymax, ymin = (a + b) / 2 + (xmax - xmin) / 2, (a + b) / 2 - (xmax - xmin) / 2

    elif (ymax - ymin) > (xmax - xmin):
      a, b = xmax, xmin
      xmax, xmin = (a + b) / 2 + (ymax - ymin) / 2, (a + b) / 2 - (ymax - ymin) / 2

    if isinstance(xmax, int) == 0: xmax = int(math.ceil(xmax))
    if isinstance(xmin, int) == 0: xmin = int(math.ceil(xmin))
    if isinstance(ymax, int) == 0: ymax = int(math.ceil(ymax))
    if isinstance(ymin, int) == 0: ymin = int(math.ceil(ymin)) 

    return xmin, ymin, xmax, ymax

In [ ]:
df = pd.read_json(annotations_dir)
files = os.listdir(files_dir)
files.sort

fn_dir = files_dir + '_fn'
fp_dir = files_dir + '_fp'


# comment for only fp detection:
neg_img_transform = transforms.Compose([                                
    transforms.Resize((128)),
    transforms.ToTensor()
])
os.makedirs(fn_dir, exist_ok = True)
for f in os.listdir(fn_dir):
    os.remove(os.path.join(fn_dir, f))

   
# comment for only fn detection:
os.makedirs(fp_dir, exist_ok=True)
for f in os.listdir(fp_dir):
    os.remove(os.path.join(fp_dir, f))
   

false_negatives = []
false_positives = []

nr_of_crops = 3

empty_imgs = 0
non_empty_imgs = 0

for i, imgdir in enumerate(tqdm(files)):
 title = files[i]    
#  title = "C:/Users/justa/Desktop/Drone/Anti-UAV CVPR 2020/img_ir/" + files [i]
 try:
    bboxes = get_square_bboxes(df[title]['bboxes'][0])
    fulldir = os.path.join(files_dir, imgdir)
    img = Image.open(fulldir)
    img = img.crop(bboxes)
    non_empty_imgs += 1
   
    # comment for only fp detection:
    if len(img.getbands()) in [1, 4]: img = img.convert('RGB')
    trans_img = neg_img_transform(img)
    if not classifier.classify_image(trans_img, model, device):
      save_image(trans_img, fn_dir+'/%d.jpg'% i)
      false_negatives.append(trans_img)
       
       
 except:
    fulldir = os.path.join(files_dir, imgdir)
    img = Image.open(fulldir)
    empty_imgs += 1
   
    # comment for only fn detection:
    if len(img.getbands()) in [1, 4]: img = img.convert('RGB')
    for x in range(nr_of_crops):
      pos_img_transform = transforms.Compose([
        transforms.RandomCrop(randint(32,256)),
        transforms.RandomHorizontalFlip(),
        transforms.Lambda(lambda crop: crop if crop.size[0] > 128 else transforms.Resize((128,128))(crop)),
        transforms.ToTensor()
      ])
      trans_img = pos_img_transform(img)
      if classifier.classify_image(trans_img, model, device):
          save_image(trans_img, fp_dir+'/%d.jpg'% i)
          false_positives.append(trans_img)
   
nr_results = len(files)

# comment for only fp detection
fnl = len(false_negatives)
print(fnl, " false negatives, accuracy = " + str(100*(1-fnl/(nr_results - empty_imgs)))+"%")
a = open(fn_dir + "/0_accuracy.txt", "w+")
a.write(str(fnl) + " false negatives, accuracy = " + str(100*(1-fnl/(nr_results - empty_imgs)))+"%\n")
a.close()


# comment for only fn detection
fpl = len(false_positives)
print(fpl, " false positives, accuracy = " + str(100*(1-fpl/(nr_of_crops*(nr_results - non_empty_imgs))))+"%")
b = open(fp_dir + "/0_accuracy.txt", "w+")
b.write(str(fpl) + " false positives, accuracy = " + str(100*(1-fpl/(nr_of_crops*(nr_results - non_empty_imgs))))+"%\n")
b.close()

len(files) == empty_imgs + non_empty_imgs

  5%|▌         | 1143/22146 [02:09<20:54, 16.74it/s] 

In [ ]:
# f = plt.figure(figsize=(8, 6))
# for j in range(6):
#     plt.subplot(2, 3, j+1)
#     plt.tight_layout()
#     plt.imshow(np.transpose(false_negatives[randint(0, fnl-1)], (1, 2, 0)))
# plt.show()

In [ ]:
# false negatives

annotations_dir = 'C:/Users/pv/Desktop/BPTI/datasets/Anti-UAV CVPR 2020/annotations_IR.json'
files_dir = 'C:/Users/pv/Desktop/BPTI/datasets/Anti-UAV CVPR 2020/img_ir'

img_transform = transforms.Compose([                                
    transforms.Resize((128)),
    transforms.ToTensor()
])

fn_dir = files_dir + '_fn'

os.makedirs(fn_dir, exist_ok = True)

for f in os.listdir(fn_dir):
    os.remove(os.path.join(fn_dir, f))
    
df = pd.read_json(annotations_dir)
files = os.listdir(files_dir)
files.sort

false_negatives = []
nr_of_crops = 1

empty_imgs = 0
imgs_without_annotation = []

for i, imgdir in enumerate(tqdm(files)):
 try:
     title = "C:/Users/justa/Desktop/Drone/Anti-UAV CVPR 2020/img_ir/" + files[i]
     if title in df:
        if df[title]['bboxes'] not in [[None,None,None,None],None,[],float('nan'),[0,0,0,0]]:
          bboxes = df[title]['bboxes'][0]
          fulldir = os.path.join(files_dir, imgdir)
          img = Image.open(fulldir)
          img = img.crop(get_square_bboxes(bboxes))
          if len(img.getbands()) in [1, 4]: img = img.convert('RGB')
          trans_img = img_transform(img)
          if not classifier.classify_image(trans_img, model, device):
              save_image(trans_img, fn_dir+'/%d.jpg'% i)
              false_negatives.append(trans_img)
        else:
            empty_imgs +=1
     else:
        imgs_without_annotation.append(files[i])
        pass
 except:
     print(files[i])
     imgs_without_annotation.append(files[i])
     pass 

nr_results = len(files) - empty_imgs - len(imgs_without_annotation)
fnl = len(false_negatives)
print(fnl, " false negatives, accuracy = " + str(100*(1-fnl/(nr_of_crops*nr_results)))+"%")

f = plt.figure(figsize=(8, 6))
for j in range(6):
    plt.subplot(2, 3, j+1)
    plt.tight_layout()
    plt.imshow(np.transpose(false_negatives[randint(0, fnl-1)], (1, 2, 0)))
plt.show()

t = open(fn_dir + "/0_accuracy.txt", "w+")
t.write(str(fnl) + " false negatives, accuracy = " + str(100*(1-fnl/(nr_of_crops*nr_results)))+"%\n")
t.write(str('\nimgs without annotations:\n'))
for i in range(len(imgs_without_annotation)):
    t.write(str(imgs_without_annotation[i]) + '\n')
t.close()

print(str(empty_imgs) + ' empty imgs')
print(str(len(imgs_without_annotation)) + ' imgs without annotations')

In [ ]:
# false positives
annotations_dir = 'C:/Users/pv/Desktop/BPTI/datasets/Anti-UAV CVPR 2020/annotations_IR.json'
files_dir = 'C:/Users/pv/Desktop/BPTI/datasets/Anti-UAV CVPR 2020/img_ir'

fp_dir = files_dir + '_fp'
os.makedirs(fp_dir, exist_ok=True)

for f in os.listdir(fp_dir):
    os.remove(os.path.join(fp_dir, f))

df = pd.read_json(annotations_dir)
files = os.listdir(files_dir)
files.sort

false_positives = []
nr_of_crops = 7

non_empty_imgs = 0
imgs_without_annotation = []
for i, imgdir in enumerate(tqdm(files)):
  if files[i] in df:
    if df[files[i]]['bboxes'] in [[None, None, None, None], [None]]:
      fulldir = os.path.join(files_dir, imgdir)
      img = Image.open(fulldir)
      if len(img.getbands()) in [1, 4]: img = img.convert('RGB')
      for x in range(nr_of_crops):
          img_transform = transforms.Compose([
            transforms.RandomCrop(randint(32,256)),
            transforms.RandomHorizontalFlip(),
            transforms.Lambda(lambda crop: crop if crop.size[0] > 128 else transforms.Resize((128,128))(crop)),
            transforms.ToTensor()
          ])
          trans_img = img_transform(img)
          if classifier.classify_image(trans_img, model, device):
              save_image(trans_img, fp_dir+'/%d.jpg'% i)
              false_positives.append(trans_img)
    else:
        non_empty_imgs += 1
 else:
    imgs_without_annotation.append(files[i])
    fulldir = os.path.join(files_dir, imgdir)
    img = Image.open(fulldir)
    if len(img.getbands()) in [1, 4]: img = img.convert('RGB')
    for x in range(nr_of_crops):
      img_transform = transforms.Compose([
        transforms.RandomCrop(randint(32,256)),
        transforms.RandomHorizontalFlip(),
        transforms.Lambda(lambda crop: crop if crop.size[0] > 128 else transforms.Resize((128,128))(crop)),
        transforms.ToTensor()
      ])
      trans_img = img_transform(img)
      if classifier.classify_image(trans_img, model, device):
          save_image(trans_img, fp_dir+'/%d.jpg'% i)
          false_positives.append(trans_img)

nr_results = len(files) - non_empty_imgs - len(imgs_without_annotation)
fpl = len(false_positives)
print(fpl, " false positives, accuracy = " + str(100*(1-fpl/(nr_of_crops*nr_results)))+"%")

f = plt.figure(figsize=(8, 6))
for i in range(6):
    plt.subplot(2, 3, i+1)
    plt.tight_layout()
    plt.imshow(np.transpose(false_positives[randint(0, fpl-1)], (1, 2, 0)))
plt.show()

t = open(fp_dir + "/0_accuracy.txt", "w+")
t.write(str(fpl) + " false positives, accuracy = " + str(100*(1-fpl/(nr_of_crops*nr_results)))+"%\n")
t.write(str('\nimgs without annotations:\n'))
for i in range(len(imgs_without_annotation)):
    t.write(str(imgs_without_annotation[i]) + '\n')
t.close()

print(str(non_empty_imgs) + 'non-empty imgs')